The Battle of Neighborhoods (Week 1)

The data that will be used to solve the problem:
We will be using Foursquare and geopy data to map top the top venues within the location of Houstons Energy Corridor 
to indicate what venues already exist in this popular area. We will create a map of the area covering about 5000m.

In [31]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [32]:
# Find the geographical coordinates of The Energy Corridor, Houston, TX
address = 'energy corridor, Houston, texas'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the Energy Corridor, Houston, TX are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the Energy Corridor, Houston, TX are 29.7862905, -95.6581257.


In [33]:
neighborhood_latitude=29.757187
neighborhood_longitude=-95.392313

In [34]:
CLIENT_ID = 'SXUPV3IPG1VRYAIEKCEIIXE4GR1ZGSUNIPTKXB5C02I514K0' # my Foursquare ID
CLIENT_SECRET = 'LC1PKVJZGO5RSMWKGYQL5JGSYC1FT14SW1OZM11FGB51GLBB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SXUPV3IPG1VRYAIEKCEIIXE4GR1ZGSUNIPTKXB5C02I514K0
CLIENT_SECRET:LC1PKVJZGO5RSMWKGYQL5JGSYC1FT14SW1OZM11FGB51GLBB


Get the top 100 venues that are near Houston's Energy Corridor. within a radius of 5000 meters

In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SXUPV3IPG1VRYAIEKCEIIXE4GR1ZGSUNIPTKXB5C02I514K0&client_secret=LC1PKVJZGO5RSMWKGYQL5JGSYC1FT14SW1OZM11FGB51GLBB&v=20180605&ll=29.7862905,-95.6581257&radius=5000&limit=100'

Send the GET request and examine the resutls

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8a4b069fb6b70a603052a1'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-505363dfe4b09a662ca2487f-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/golfcourse_',
          'suffix': '.png'},
         'id': '58daa1558bbb0b01f18ec1b0',
         'name': 'Golf Driving Range',
         'pluralName': 'Golf Driving Ranges',
         'primary': True,
         'shortName': 'Golf Driving Range'}],
       'id': '505363dfe4b09a662ca2487f',
       'location': {'address': '1030 Memorial Brook Blvd',
        'cc': 'US',
        'city': 'Houston',
        'country': 'United States',
        'distance': 781,
        'formattedAddress': ['1030 Memorial Brook Blvd',
         'Houston, TX 77084',
         'United States'],
        'labele

In [36]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Topgolf,Golf Driving Range,29.786743,-95.650050
1,Brother's Pizzeria,Pizza Place,29.786889,-95.645525
2,Watson's House of Ale's,Pub,29.782844,-95.643579
3,Lupe Tortilla Mexican Restaurant,Mexican Restaurant,29.782903,-95.643100
4,North China Restaurant,Chinese Restaurant,29.783944,-95.642419
5,Salata,Salad Place,29.783908,-95.641992
6,SpringHill Suites by Marriott,Hotel,29.787306,-95.658506
7,Christian's Tailgate Bar & Grill,Sports Bar,29.786865,-95.644110
8,Annam Indian Cuisine,Indian Restaurant,29.786948,-95.645506
9,Fairfield Inn & Suites Houston I-10 West/Energ...,Hotel,29.783947,-95.650437


In [40]:
# create map of Houston's Energy Corridor using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg